<a href="https://colab.research.google.com/github/franconoronha/treinamento-h2ia/blob/main/2_BuscasSemInforma%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

Acabei por não utilizar o numpy, pois na minha cabeça fazia mais sentido representar os estados utilizando strings, já que pelos meus testes ocuparia menos memória. Porém não sei se vale a pena, visto que os arrays do numpy foram feitos para ter boa perfomance, e as strings do python são imutáveis e por isso dificultam um pouquinho a manipulação (ou seja provavelmente mais lento), no fim acho que o ganho na memória não foi tão significativo, quem sabe para tamanhos maiores de quebra-cabeças. 

Também optei por fazer tudo de forma iterativa e não recursiva, o python não curtiu muito a recursão, mas não sei se isso é algo que deveria ignorar no futuro.

Comparação de memória:

In [ ]:
import numpy as np
from sys import getsizeof
string = "123456789"
np_array = np.array([[1,2,3],[4,5,6],[7,8,9]])

print("Tamanho de um string de 9 char: {} bytes".format(getsizeof(string)))
print("Tamanho de uma matriz numpy 3x3: {} bytes".format(getsizeof(np_array)))

Tamanho de um string de 9 char: 58 bytes
Tamanho de uma matriz numpy 3x3: 192 bytes


Importação de bibliotecas, definição de funções auxiliares e geração de um puzzle "aleatório".

In [ ]:
from collections import deque
from random import seed, randint
 
# Comentar a seed para ter um puzzle aleatório
seed(101)

class Tree:
    def __init__(self, data, parent, move):
        self.children = []
        self.data = data
        self.parent = parent
        self.move = move

""" Representando o puzzle como uma string
* representa o espaço vazio
Movimentos possíveis
(considerando que quem se move é o * e ele troca de lugar com a peça pra onde tá indo)
- pra cima: -3 posições
- pra baixo: +3 
- pra direita: +1
- pra esquerda: -1

1.só é possível se (pos_atual + movimento) for maior ou igual a 0 e menor que len(puzzle)  
2.não é possível mexer para esquerda na coluna da esquerda (posições 0, 3, 6 % 3 == 0)
3.não é possível mexer para direita na coluna da direita (posições 2, 5, 8 % 3 == 2)
"""

def gerar_puzzle():
  puzzle = ""
  numeros = ["1", "2", "3", "4", "5", "6", "7", "8"]
  for x in range(0, len(numeros)):
    char = numeros.pop(randint(0, len(numeros) - 1))
    puzzle += char
  puzzle += '*'
  return puzzle

def get_movimentos_legais(puzzle):
  ind = puzzle.find("*")
  moves = []
  for x in [3, -3, 1, -1]:
    if x == -1 and ind % 3 == 0:
      continue
    elif x == 1 and ind % 3 == 2:
      continue
    if((ind + x) < len(puzzle) and (ind + x) >= 0):
      moves.append(x)
  return moves

def swap(puzzle, move):
  index = puzzle.find('*')
  move += index
  p = list(puzzle)
  p[index], p[move] = p[move], p[index]
  return ''.join(p)

def aplica_resultado(puzzle, resultado):
  if resultado == -1:
    return puzzle
  for move in resultado:
    puzzle = swap(puzzle, move)
  return puzzle
  
def nodos_arvore(arvore):
  contador = 0
  fila = deque()
  fila.append(arvore)
  while len(fila) > 0:
    atual = fila.popleft()
    fila += atual.children
    contador += 1
  return contador

solucao = "12345678*"
puzzle = gerar_puzzle()
puzzle = "8672543*1"
print("Puzzle gerado: {}".format(puzzle))

Puzzle gerado: 8672543*1


## Busca em largura

In [ ]:
def busca_largura(puzzle):
  global solucao
  borda = deque()
  visitados = set()
  contador = 0

  arvore = Tree(puzzle, None, None) # arvore de busca
  visitados.add(puzzle)             # estados gerados/"visitados"
  borda.append(arvore)

  while len(borda) > 0:
      atual = borda.popleft()
      if atual.data == solucao:
          break

      moves = get_movimentos_legais(atual.data) 
      novos_estados = [(swap(atual.data, x), x) for x in moves]

      for estado in novos_estados:
          if estado[0] not in visitados:
              contador += 1
              novo_nodo = Tree(estado[0], atual, estado[1])
              #arvore.children.append(novo_nodo) 
              borda.append(novo_nodo)
              visitados.add(estado[0]) 

  print("Estados gerados: {}".format(contador))
  if atual.data == solucao:
      print("Solucionado.")
      #print("Nodos na árvore: {}".format(nodos_arvore(arvore)))
      # N de nodos vai ser sempre igual aos estados
      moves_solucao = []
      while atual.move != None:
        moves_solucao.append(atual.move)
        atual = atual.parent
      return moves_solucao[::-1]
  else:
      print("Não existe uma solução")
      return -1

resultado_largura = busca_largura(puzzle)
print(resultado_largura)
print(aplica_resultado(puzzle, resultado_largura))

Estados gerados: 181439
Solucionado.
[-3, -3, 1, 3, -1, -1, 3, 1, 1, -3, -1, -1, -3, 1, 1, 3, -1, 3, -1, -3, -3, 1, 3, 3, -1, -3, -3, 1, 1, 3, 3]
12345678*


## Busca em Profundidade

Acho que a parte de corte não funcionou corretamente. Ele tira a relação de pai/filho dos nós (já que a função de contar os nodo retorna um número bem menor que os estados gerados), mas pelo jeito não deleta os nós de fato, não sei se é pela minha falta de familiaridade com python, mas ao meu ver parece correto, a não ser que eu esteja apenas deletando a referência para o objeto e não o objeto de fato.

((Acho que o problema era o getsizeof mesmo, pelo jeito tá funcionando... eu acho))

In [ ]:
def busca_profundidade(puzzle):
  global solucao
  contador = 0
  borda = deque()
  visitados = set()

  arvore = Tree(puzzle, None, None) # arvore de busca
  visitados.add(puzzle)       # estados gerados/"visitados"
  borda.append(arvore)

  while len(borda) > 0:
      atual = borda.pop()     # pop ao inves de popleft
      if atual.data == solucao:
          break

      moves = get_movimentos_legais(atual.data) 
      novos_estados = [(swap(atual.data, x), x) for x in moves]
      estados_validos = [x for x in novos_estados if x[0] not in visitados]

      if len(estados_validos) == 0: # Apaga os caminhos fechados
        prev = atual
        atual = atual.parent
        atual.children.remove(prev)
        del prev
        while len(atual.children) == 0 and atual.parent != None:
          prev = atual
          atual = atual.parent
          atual.children.remove(prev)
          del prev
      
      for estado in estados_validos:
        contador += 1
        novo_nodo = Tree(estado[0], atual, estado[1])
        atual.children.append(novo_nodo)
        borda.append(novo_nodo)
        visitados.add(estado[0])  

  print("Estados gerados: {}".format(contador))
  if atual.data == solucao:
      print("Solucionado.")
      print("Nodos na árvore: {}".format(nodos_arvore(arvore)))
      moves_solucao = []
      while atual.move != None:
        moves_solucao.append(atual.move)
        atual = atual.parent
      return moves_solucao[::-1]
  else:
      print("Não existe uma solução")
      return -1

resultado_profundidade = busca_profundidade(puzzle)
#print(resultado_profundidade)
print(len(resultado_profundidade))
print(aplica_resultado(puzzle, resultado_profundidade))


Estados gerados: 179094
Solucionado.
Nodos na árvore: 53165
30280
12345678*
